In [ ]:
import os; os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import detection_demo

import numpy as np

In [ ]:
image_dir = './cub200-dataset/images/'
ann_file = './data/image_ann_dict.npz'
save_dir = './data/explanation_feat/'

In [ ]:
ann_dict = np.load(ann_file)['im_name2ann'][()]
num_im = len(ann_dict)

In [ ]:
for n_im, im_name in enumerate(ann_dict):
    print('processing %d / %d' % (n_im, num_im))
    
    im_path = os.path.join(image_dir, im_name)
    save_path = os.path.join(save_dir, im_name.replace('.jpg', '.npz'))
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    anns = ann_dict[im_name]
    query_list = []
    for ann in anns:
        for phrase in ann['noun_phrases']:
            query_list.append(phrase['phrase'])
    query_list = list(set(query_list))
    print('number of unique noun phrases:', len(query_list))
    
    proposal_bboxes, scores_val, im_visfeat, bbox_visfeat = \
        detection_demo.run_on_image(im_path, query_list)
    scores_val = scores_val.reshape((len(query_list), len(proposal_bboxes)))
    
    pred_bbox = np.argmax(scores_val, axis=1)
    query_scores = [scores_val[n_q, pred_bbox[n_q]] for n_q in range(len(query_list))]
    query_visfeat = list(bbox_visfeat[pred_bbox])
    
    query_dict = {query_list[n_q]: {'score': query_scores[n_q],
                                    'visfeat': query_visfeat[n_q]}
                  for n_q in range(len(query_list))}
    
    explanation_feat = {'im_visfeat': im_visfeat, 'query_dict': query_dict}
    np.savez(save_path, explanation_feat=explanation_feat)